# SK Summarizer Pattern

Learning objectives:

- Understand a simple Summarization Implementation

## Setup

### Load required .NET packages and supporting constants, classes, etc.

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.4.0"
#r "nuget: Microsoft.SemanticKernel.Core, 1.4.0"
#r "nuget: dotenv.net"

using System;
using System.IO;
using System.Text;
using System.Text.RegularExpressions;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using dotenv.net;

#!import Models/Models.cs
#!import Utils/TextChunker.cs
#!import Templates/Templates.cs

const int ChunkSize = 1000;
const int MaxTokens = 1500;
const double Temperature = 0.3;

Installed Packages dotenv.net, 3.1.3 Microsoft.SemanticKernel, 1.4.0 Microsoft.SemanticKernel.Core, 1.4.0

### Read the API Key and endpoints from environment variables or the .env file

In [2]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
var adaDeploymentName = "ada";
var checkKeyAndEndpoint = $"Endpoint: {endpoint}{deploymentName} with key {apiKey.Substring(0, 5)}...";
checkKeyAndEndpoint

Endpoint: https://alemorfr.openai.azure.com/gpt with key 32c87...

### Get a Semantic Kernel instance configured for Chat completions

In [3]:
var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(deploymentName, endpoint, apiKey)
    .Build();

## Ingestion

### Read transcript from a file

In [4]:
var text = File.ReadAllText("data/transcript.txt");
text

[Teams Call Transcript]

Participants:
1. John - Azure Architect
2. Sarah - DevOps Engineer
3. Mark - Software Developer

[Call Start]

John: Good morning, everyone. Thank you for joining this call. Today, we'll discuss running Docker applications on Azure Virtual Machines. Sarah, could you provide an overview of the current setup?

Sarah: Sure, John. Currently, we have a few Azure Virtual Machines (VMs) provisioned, and we want to leverage Docker to run our applications on these VMs. We have already installed Docker on the VMs, but we need to ensure that our applications are running smoothly and efficiently.

Mark: Right, Sarah. I have been working on containerizing our applications using Docker Compose. I believe this will make it easier to manage our services and scale them when needed.

John: Excellent, Mark. Docker Compose is a great choice for managing multi-container applications. Now, let's discuss how we can optimize the deployment of these Dockerized applications on Azure VMs

## Chunk the text

In [5]:
var chunks = TextChunker.ChunkText(text, ChunkSize);
chunks

Chunking text into chunks of max size: 1000
Chunked text into 2 chunk(s)


index value 0 Submission#3+ChunkInfo Index 0 Content [Teams Call Transcript]

Participants:
1. John - Azure Architect
2. Sarah - DevOps Engineer
3. Mark - Software Developer

[Call Start]

John: Good morning, everyone. Thank you for joining this call. Today, we'll discuss running Docker applications on Azure Virtual Machines. Sarah, could you provide an overview of the current setup?

Sarah: Sure, John. Currently, we have a few Azure Virtual Machines (VMs) provisioned, and we want to leverage Docker to run our applications on these VMs. We have already installed Docker on the VMs, but we need to ensure that our applications are running smoothly and efficiently.

Mark: Right, Sarah. I have been working on containerizing our applications using Docker Compose. I believe this will make it easier to manage our services and scale them when needed.

John: Excellent, Mark. Docker Compose is a great choice for managing multi-container applications. Now, let's discuss how we can optimize the deployment of these Dockerized applications on Azure VMs. One option is to use Azure Container Instances (ACI) directly. However, since we have VMs already provisioned, it would be more cost-effective to use them.

Sarah: Agreed, John. So, what steps do we need to take to ensure the VMs are properly configured for Docker?

John: Firstly, we need to make sure that the VMs have the necessary Docker runtime installed. You mentioned that Docker is already installed, but let's ensure we have the latest version. We can do this by running the command "docker --version" on each VM.

Mark: I'll take care of that. I'll log in to each VM and check the Docker version. If needed, I'll update it to the latest stable release.

John: Great, Mark. In addition to the Docker runtime, we should also check the VM's resources. We need to ensure that each VM has enough CPU, memory, and storage to handle the expected workload of the Dockerized applications.

Sarah: Understood. We can check the VM sizes and adjust them accordingly. If we anticipate high traffic or resource-intensive applications, we may need to upscale the VMs.

John: Exactly, Sarah. It's important to monitor the performance of the VMs once we deploy the Docker applications. We can use Azure Monitor to collect and analyze metrics like CPU usage, memory utilization, and disk I/O.

Mark: That sounds good. I'll make a note to set up Azure Monitor for our VMs. We can set up alerts based on specific thresholds to get notified if any resource is running low.

John: Excellent, Mark. Additionally, we should consider deploying our Dockerized applications behind a load balancer to distribute the traffic evenly among the VMs. Azure Load Balancer or Azure Application Gateway can serve this purpose.

Sarah: Right, a load balancer will ensure high availability and scalability. We can configure it to distribute traffic based on various algorithms, such as round-robin or least connections.

John: Absolutely. Lastly, let's discuss security. We should ensure that the VMs running Docker applications are properly secured. We can leverage Azure Security Center to monitor and detect any vulnerabilities or suspicious activities.

Mark: Security is crucial. I'll make sure we enable Azure Security Center and configure it to provide recommendations and alerts for our VMs.

John: That's great, Mark. Okay, I think we have a good plan in place. Mark, please update the Docker runtime on the VMs, Sarah, adjust the VM sizes if necessary, and Mark, take care of setting up Azure Monitor, load balancer, and Azure Security Center. Let's reconvene next week to discuss any challenges or progress we've made.

Sarah: Sounds like a plan, John.

[Call Continues]

John: Perfect. I appreciate your commitment, Sarah and Mark. If you come across any issues or need assistance during the implementation process, don't hesitate to reach out to me or the team for support.

Mark: Sure, John. I'll keep you updated on the progress and let you know if any hurdl

## Process the chunk summaries

In [6]:
foreach(var chunk in chunks)
{
    // REMINDER: Templates are in the Templates/Templates.cs file
    var prompt = SummaryTemplate.Replace("{context}", chunk.Content);
    // Create a fixed SK function
    var fixedFunction = kernel.CreateFunctionFromPrompt(prompt);
    // Run the function
    var result = await kernel.InvokeAsync(fixedFunction);
    chunk.Summary = result.ToString();
}
chunks

index value 0 Submission#3+ChunkInfo Index 0 Content [Teams Call Transcript]

Participants:
1. John - Azure Architect
2. Sarah - DevOps Engineer
3. Mark - Software Developer

[Call Start]

John: Good morning, everyone. Thank you for joining this call. Today, we'll discuss running Docker applications on Azure Virtual Machines. Sarah, could you provide an overview of the current setup?

Sarah: Sure, John. Currently, we have a few Azure Virtual Machines (VMs) provisioned, and we want to leverage Docker to run our applications on these VMs. We have already installed Docker on the VMs, but we need to ensure that our applications are running smoothly and efficiently.

Mark: Right, Sarah. I have been working on containerizing our applications using Docker Compose. I believe this will make it easier to manage our services and scale them when needed.

John: Excellent, Mark. Docker Compose is a great choice for managing multi-container applications. Now, let's discuss how we can optimize the deployment of these Dockerized applications on Azure VMs. One option is to use Azure Container Instances (ACI) directly. However, since we have VMs already provisioned, it would be more cost-effective to use them.

Sarah: Agreed, John. So, what steps do we need to take to ensure the VMs are properly configured for Docker?

John: Firstly, we need to make sure that the VMs have the necessary Docker runtime installed. You mentioned that Docker is already installed, but let's ensure we have the latest version. We can do this by running the command "docker --version" on each VM.

Mark: I'll take care of that. I'll log in to each VM and check the Docker version. If needed, I'll update it to the latest stable release.

John: Great, Mark. In addition to the Docker runtime, we should also check the VM's resources. We need to ensure that each VM has enough CPU, memory, and storage to handle the expected workload of the Dockerized applications.

Sarah: Understood. We can check the VM sizes and adjust them accordingly. If we anticipate high traffic or resource-intensive applications, we may need to upscale the VMs.

John: Exactly, Sarah. It's important to monitor the performance of the VMs once we deploy the Docker applications. We can use Azure Monitor to collect and analyze metrics like CPU usage, memory utilization, and disk I/O.

Mark: That sounds good. I'll make a note to set up Azure Monitor for our VMs. We can set up alerts based on specific thresholds to get notified if any resource is running low.

John: Excellent, Mark. Additionally, we should consider deploying our Dockerized applications behind a load balancer to distribute the traffic evenly among the VMs. Azure Load Balancer or Azure Application Gateway can serve this purpose.

Sarah: Right, a load balancer will ensure high availability and scalability. We can configure it to distribute traffic based on various algorithms, such as round-robin or least connections.

John: Absolutely. Lastly, let's discuss security. We should ensure that the VMs running Docker applications are properly secured. We can leverage Azure Security Center to monitor and detect any vulnerabilities or suspicious activities.

Mark: Security is crucial. I'll make sure we enable Azure Security Center and configure it to provide recommendations and alerts for our VMs.

John: That's great, Mark. Okay, I think we have a good plan in place. Mark, please update the Docker runtime on the VMs, Sarah, adjust the VM sizes if necessary, and Mark, take care of setting up Azure Monitor, load balancer, and Azure Security Center. Let's reconvene next week to discuss any challenges or progress we've made.

Sarah: Sounds like a plan, John.

[Call Continues]

John: Perfect. I appreciate your commitment, Sarah and Mark. If you come across any issues or need assistance during the implementation process, don't hesitate to reach out to me or the team for support.

Mark: Sure, John. I'll keep you updated on the progress and let you know if any hurdl

## Final summary

In [7]:
var augmentedContext = new StringBuilder();

foreach(var chunk in chunks)
{
    augmentedContext.Append(chunk.Summary+"\n\n");    
}

// REMINDER: Templates are in the Templates/Templates.cs file
var funcTemplate = FinalSummaryTemplate.Replace("{context}", augmentedContext.ToString());

// Create a Prompt SK function from the template
var fixedFunction = kernel.CreateFunctionFromPrompt(funcTemplate,new OpenAIPromptExecutionSettings() { MaxTokens = MaxTokens, Temperature = Temperature, TopP = 1 });

// Run the function
var result = await kernel.InvokeAsync(fixedFunction);
result.ToString()

Summary:
The team discussed the necessary steps for optimizing the deployment of Dockerized applications on Azure Virtual Machines. They talked about the necessary configurations of the VMs for Docker, monitoring performance, load balancing, Azure Security Center, and backup mechanisms. AKS was also explored for future scalability considerations. In addition, they mentioned updating the Docker runtime, adjusting VM sizes, setting up Azure Monitor, load balancer, and Azure Security Center, implementing a reliable backup strategy, and keeping AKS in mind for future scalability considerations.

Azure services:
- Azure Virtual Machines
- Azure Container Instances (ACI)
- Azure Monitor
- Azure Load Balancer or Azure Application Gateway
- Azure Security Center
- Azure Backup
- Azure Kubernetes Service (AKS)

Other technologies:
- Docker Compose

Action items:
1. Mark to update the Docker runtime on the VMs
2. Sarah to adjust the VM sizes if necessary
3. Mark to set up Azure Monitor, load bal